In [ ]:
#Librería para manipular datos
import pandas as pd
# llamar a la API de Zendesk
import numpy as np
import httplib2
import time
import json
import requests
import base64
import glob
import os
from os import walk

from google.colab import drive
drive.mount('/content/drive')

#config
pd.set_option('display.max_columns', None)

In [ ]:
#_______________________________________________________________________________________
z_domain=""
email=""
contraseña=""
start_time="946684800"
#start_time="1651363200"
#start_time="1660660764"
carpeta_contenedora="/content/drive/MyDrive/Colab Notebooks/"
#_______________________________________________________________________________________




In [ ]:
authentication=email+"/token:"+contraseña
message_bytes = authentication.encode('ascii')
base64_bytes=base64.b64encode(message_bytes).decode('ascii')
headers = {'Authorization': 'Basic '+base64_bytes, 'Content-Type': 'application/json'}


In [ ]:

def data_ticket(z_domain,start_time,headers):
  url = z_domain+"/api/v2/incremental/tickets.json?start_time="   +start_time+   "&include=metric_sets,organizations,users"
  status = 0
  while status != 200:
    r = requests.get(url, headers=headers)
    
    #response, content = http.request(url, 'GET', headers=headers)
    status = r.status_code
    if status == 200:
      print("----llamada de api exitosa: ", status)
      print("----respuesta: ", r.text)
    else:
      print("----llamada de api fracasada: ", status)
      print("----respuesta: ", r.text)
      time.sleep(20)

  data=json.loads(r.text)
  is_cont=data['end_of_stream']
  end_time=data['end_time']

  df_ticket=pd.DataFrame(data['tickets'])

  all_cf=pd.DataFrame()
  # Se iteran todos los valores de cada ticket devuelto, pueden ser hasta 1000
  for idx, x in enumerate(df_ticket['id']):
    #Purga e instancia estos arreglos
    custom_v_col=[]
    custom_v_val=[]
    #Purga e instancia el dataset
    df_cf=pd.DataFrame()
    c_val=df_ticket['custom_fields'][idx]
    for idc, c in enumerate(c_val):
      #Registra en 2 arreglos, los ID que serán usandos como columnas y los valores como elementos de cada fila
      custom_v_col.append(str(c['id']))
      custom_v_val.append(str(c['value']))
    # Crea la fila del ticket
    df_cf=pd.DataFrame([custom_v_val], columns=custom_v_col)
    # Agrega el canal por donde llego el ticket
    df_cf['canal ingreso']=df_ticket['via'][idx]['channel']
    # concatena las nueva filas con otras creadas anteriormente
    all_cf = pd.concat([all_cf, df_cf])
    

  #Reinicia el index del arreglo
  all_cf=all_cf.reset_index()
  # se unen los 2 dataset
  df_ticket = pd.concat([df_ticket, all_cf], axis=1)

  #se traspasa las variables del satisfaction_rating
  score=[]
  comment=[]
  reason=[]
  for i in df_ticket['satisfaction_rating']:
    
   if i['score'] == "unoffered":
     score.append("No se ha ofrecido")
   elif i['score'] == "offered":
     score.append("Se ha ofrecido")
   elif i['score'] == "good":
     score.append(i['score'])
   elif i['score'] == "bad":
     score.append(i['score'])

   try:
     comment.append(i['comment'])
   except:
     comment.append("no hay comentarios")
   try:
     reason.append(i['comment'])
   except:
     reason.append("No se proporcionó una razón")
  df_ticket['indice de satisfaccion']=score
  df_ticket['comentario satisfaccion']=comment
  df_ticket['motivos satisfaccion']=comment

  #se traspasa las variables del metric_set
  group_stations=[]
  assignee_stations=[]
  reopens=[]
  replies=[]
  assignee_updated_at=[]
  requester_updated_at=[]
  status_updated_at=[]
  initially_assigned_at=[]
  assigned_at=[]
  solved_at=[]
  latest_comment_added_at=[]
  reply_time_in_minutes_calendar=[]
  reply_time_in_minutes_business=[]
  first_resolution_time_in_minutes_calendar=[]
  first_resolution_time_in_minutes_business=[]
  full_resolution_time_in_minutes_calendar=[]
  full_resolution_time_in_minutes_business=[]
  agent_wait_time_in_minutes_calendar=[]
  agent_wait_time_in_minutes_business=[]
  requester_wait_time_in_minutes_calendar=[]
  requester_wait_time_in_minutes_business=[]
  on_hold_time_in_minutes_calendar=[]
  on_hold_time_in_minutes_business=[]

  for index, i in enumerate(df_ticket['metric_set']):
    try:
      group_stations.append(i['group_stations'])
      assignee_stations.append(i['assignee_stations'])
      reopens.append(i['reopens'])
      replies.append(i['replies'])
      assignee_updated_at.append(i['assignee_updated_at'])
      requester_updated_at.append(i['requester_updated_at'])
      status_updated_at.append(i['status_updated_at'])
      initially_assigned_at.append(i['initially_assigned_at'])
      assigned_at.append(i['assigned_at'])
      solved_at.append(i['solved_at'])
      latest_comment_added_at.append(i['latest_comment_added_at'])
      reply_time_in_minutes_calendar.append(i['reply_time_in_minutes']['calendar'])
      reply_time_in_minutes_business.append(i['reply_time_in_minutes']['business'])
      first_resolution_time_in_minutes_calendar.append(i['first_resolution_time_in_minutes']['calendar'])
      first_resolution_time_in_minutes_business.append(i['first_resolution_time_in_minutes']['business'])
      full_resolution_time_in_minutes_calendar.append(i['full_resolution_time_in_minutes']['calendar'])
      full_resolution_time_in_minutes_business.append(i['full_resolution_time_in_minutes']['business'])
      agent_wait_time_in_minutes_calendar.append(i['agent_wait_time_in_minutes']['calendar'])
      agent_wait_time_in_minutes_business.append(i['agent_wait_time_in_minutes']['business'])
      requester_wait_time_in_minutes_calendar.append(i['requester_wait_time_in_minutes']['calendar'])
      requester_wait_time_in_minutes_business.append(i['requester_wait_time_in_minutes']['business'])
      on_hold_time_in_minutes_calendar.append(i['on_hold_time_in_minutes']['calendar'])
      on_hold_time_in_minutes_business.append(i['on_hold_time_in_minutes']['business'])
    except:
      group_stations.append(None)
      assignee_stations.append(None)
      reopens.append(None)
      replies.append(None)
      assignee_updated_at.append(None)
      requester_updated_at.append(None)
      status_updated_at.append(None)
      initially_assigned_at.append(None)
      assigned_at.append(None)
      solved_at.append(None)
      latest_comment_added_at.append(None)
      reply_time_in_minutes_calendar.append(None)
      reply_time_in_minutes_business.append(None)
      first_resolution_time_in_minutes_calendar.append(None)
      first_resolution_time_in_minutes_business.append(None)
      full_resolution_time_in_minutes_calendar.append(None)
      full_resolution_time_in_minutes_business.append(None)
      agent_wait_time_in_minutes_calendar.append(None)
      agent_wait_time_in_minutes_business.append(None)
      requester_wait_time_in_minutes_calendar.append(None)
      requester_wait_time_in_minutes_business.append(None)
      on_hold_time_in_minutes_calendar.append(None)
      on_hold_time_in_minutes_business.append(None)

  df_ticket['grupos asignados']=group_stations
  df_ticket['agentes asignados']=assignee_stations
  df_ticket['reaperturas']=reopens
  df_ticket['cantidad de respuestas']=replies
  df_ticket['ultima actualizacion agente']=assignee_updated_at
  df_ticket['ultima actualizacion solicitante']=requester_updated_at
  df_ticket['ultima actualizacion estado']=status_updated_at
  df_ticket['inicialmente asignado']=initially_assigned_at
  df_ticket['fecha asignacion']=assigned_at
  df_ticket['ultimo comentario agregado']=latest_comment_added_at
  df_ticket['tiempo respuesta calendario']=reply_time_in_minutes_calendar
  df_ticket['tiempo respuesta negocio']=reply_time_in_minutes_business
  df_ticket['primera resolucion calendario']=first_resolution_time_in_minutes_calendar
  df_ticket['primera resolucion negocio']=first_resolution_time_in_minutes_business
  df_ticket['resolucion completa calendario']=full_resolution_time_in_minutes_calendar
  df_ticket['resolucion completa negocio']=full_resolution_time_in_minutes_business
  df_ticket['espera agente calendario']=agent_wait_time_in_minutes_calendar
  df_ticket['espera agente negocio']=agent_wait_time_in_minutes_business
  df_ticket['espera solicitante calendario']=requester_wait_time_in_minutes_calendar
  df_ticket['espera solicitante negocio']=requester_wait_time_in_minutes_business
  df_ticket['tiempo espera calendario']=on_hold_time_in_minutes_calendar
  df_ticket['tiempo espera negocio']=on_hold_time_in_minutes_business

  return df_ticket, is_cont, end_time

In [ ]:
#a,b,c=data_ticket(z_domain,start_time,headers)
#a

In [ ]:
def data_user(z_domain,start_time,headers):
  continua2=True
  continua=True
  org_time=str(946684800)
  user_time=str(946684800)
  all_org={}
  iter=1
  print("---Lo lamentamos, esto puede demorar")
  while continua2:
    print("---extración de info org iter "+str(iter)+",COD "+org_time)
    url2 = z_domain+"/api/v2/incremental/organizations.json?start_time="+org_time
    http = httplib2.Http()
    response, content = http.request(url2, 'GET', headers=headers)
    data2=json.loads(content)
    time.sleep(15)
    #print(data2)
    #print("_________")
    for idx, x in enumerate(data2['organizations']):
      all_org[str(x['id'])]=[x['name']]

    if(data2['end_of_stream']):
      continua2=False 
    org_time=str(data2['end_time'])
  print("---Se obtuvo la información de las organizaciones")
  #__________________________________________________________________________________________________
  all_user={}
  iter=1
  while continua:
    print("---extración de info usuario iter "+str(iter)+",COD "+user_time)
    url = z_domain+"/api/v2/incremental/users.json?start_time="+user_time
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(15)
    #print(data)
    #print("_________")
    for idx, x in enumerate(data['users']):
      if x['organization_id'] is not None:
        nom_org=all_org[str(x['organization_id'])][0]
      else:
        nom_org=""
      all_user[str(x['id'])]=[x['name'],x['email'],str(nom_org)]

    if(data['end_of_stream']):
      continua=False 
    user_time=str(data['end_time'])
    iter=iter+1
  print("---Se obtuvo la información de los usuarios")

  return all_user


In [ ]:
#a2= data_user(z_domain,start_time,headers)

In [ ]:
#a2

In [ ]:
#os.chdir(carpeta_contenedora)
#file_extension = '.csv'
#all_filenames = [i for i in glob.glob(f"*{file_extension}")]
#all_filenames
#combined_csv_data = pd.concat([pd.read_csv(f, encoding='UTF-8') for f in all_filenames])

In [ ]:
def generarDataSetCamposTicket(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_fields"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    campos_de_ticket={}
    for idx, x in enumerate(data['ticket_fields']):
        id =  data['ticket_fields'][idx]['id']
        nombreCampo =  data['ticket_fields'][idx]['title']
        type =  data['ticket_fields'][idx]['type']


        if type == 'tagger' or type == 'multiselect':
            custom_field_options = data['ticket_fields'][idx]['custom_field_options']
            opciones_campo = []
            for idy, y in enumerate(custom_field_options):
                opciones_campo.append([custom_field_options[idy]['raw_name'], custom_field_options[idy]['value']])

            campos_de_ticket[id]=[nombreCampo, type, opciones_campo]

        if type == 'checkbox':
            tag = custom_field_options = data['ticket_fields'][idx]['tag']
            campos_de_ticket[id]=[nombreCampo, type, tag]

        if type != 'checkbox' and type != 'tagger' and type != 'multiselect':
            campos_de_ticket[id]=[nombreCampo, type]

    
    return campos_de_ticket

In [ ]:
#lista_campos=generarDataSetCamposTicket(z_domain,start_time,headers)
#lista_campos

In [ ]:
def generarDataSetFormularios(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/ticket_forms"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    formularios={}
    for idx, x in enumerate(data['ticket_forms']):
        id = data['ticket_forms'][idx]['id'];
        nombre = data['ticket_forms'][idx]['name'];
        formularios[id] = nombre
        
    return formularios

In [ ]:
#generarDataSetFormularios(z_domain,start_time,headers)

In [ ]:
def generarDataSetAgentes():    
    url = z_domain+"/api/v2/users?role=agent&role=admin"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)

    agentes={}
    for idx, x in enumerate(data['users']):
        id = data['users'][idx]['id'];
        nombre = data['users'][idx]['name'];
        agentes[id] = nombre
        
    return agentes

In [ ]:
def generarDataSetMarcas(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/brands"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    marcas={}
    for idx, x in enumerate(data['brands']):
        id = data['brands'][idx]['id'];
        nombre = data['brands'][idx]['name'];
        marcas[id] = nombre
        
    return marcas

In [ ]:
def generarDataGrupo(z_domain,start_time,headers):    
    url = z_domain+"/api/v2/groups"
    http = httplib2.Http()
    response, content = http.request(url, 'GET', headers=headers)
    data=json.loads(content)
    time.sleep(4)
    grupo={}
    for idx, x in enumerate(data['groups']):
        id = x['id']
        nombre = x['name']
        grupo[id] = nombre
        
    return grupo

In [ ]:
def etl_ticket(z_domain,start_time,headers,carpeta_contenedora):
  cont=True
  list_csv=[]
  N_1=1
  print("Estamos extrayendo las referencias para este ETL")
  print("-Descargando data formularios")
  lista_formulario=generarDataSetFormularios(z_domain,start_time,headers)
  print("--Cantidad",len(lista_formulario))
  print("-Descargando data usuarios")
  lista_usuario=data_user(z_domain,start_time,headers)
  print("--Cantidad",len(lista_usuario))
  print("-Descargando data campos personalizados")
  lista_campos=generarDataSetCamposTicket(z_domain,start_time,headers)
  print("--Cantidad de campos",len(lista_campos))
  print("-Descargando lista de marcas")
  lista_marca=generarDataSetMarcas(z_domain,start_time,headers)
  print("--Cantidad",len(lista_marca))
  print("-Descargando lista de grupos")
  lista_grupo=generarDataGrupo(z_domain,start_time,headers)
  print("--Cantidad",len(lista_grupo))
  try:
    os.remove(carpeta_contenedora+"/01.csv")
  except:
    print("No se encontro un archivo 01.csv")
  print("Parte extracción de los datos de tickets")
  while cont:
    print("_______________________________________________________")
    print("-Iteración "+str(N_1)+":",start_time, "Trabajando 😔")
    data,cont_1,new_start=data_ticket(z_domain,start_time,headers)
    print("--Se consiguío los datos de Zendesk")
    
    time.sleep(4)
    # Acá parte la sincronización del nombre de formulario
    nombre_form=[]
    for ticket in data['ticket_form_id']:
      if np.isnan(ticket):
        nombre_form.append("")
      else:
        nombre_form.append(lista_formulario[ticket])

    data['Nombre del formulario']=nombre_form
    # Información de los usuarios
    
    nombre_solicitante=[]
    correo_solicitante=[]
    organizacion_solicitante=[]
    nombre_agente=[]
    correo_agente=[]
    organizacion_agente=[]
    nombre_remitente=[]
    correo_remitente=[]
    organizacion_remitente=[]

    #solicitante_____________________________________________________
    for req in data['requester_id']:
      if np.isnan(req):
        nombre_solicitante.append("")
        correo_solicitante.append("")
        organizacion_solicitante.append("")
      else:
        try:
          nombre_solicitante.append(lista_usuario[str(int(req))][0])
          correo_solicitante.append(lista_usuario[str(int(req))][1])
          organizacion_solicitante.append(lista_usuario[str(int(req))][2])
        except:
          nombre_solicitante.append("")
          correo_solicitante.append("")
          organizacion_solicitante.append("")

    #Agente_____________________________________________________
    for agent in data['assignee_id']:
      if np.isnan(agent):
        nombre_agente.append("")
        correo_agente.append("")
        organizacion_agente.append("")
      else:
        try:
          nombre_agente.append(lista_usuario[str(int(agent))][0])
          correo_agente.append(lista_usuario[str(int(agent))][1])
          organizacion_agente.append(lista_usuario[str(int(agent))][2])
        except:
          nombre_agente.append("")
          correo_agente.append("")
          organizacion_agente.append("")

    #remitente_____________________________________________________
    for submi in data['submitter_id']:
        if np.isnan(submi):
          nombre_remitente.append("")
          correo_remitente.append("")
          organizacion_remitente.append("")
        else:
          try:
            nombre_remitente.append(lista_usuario[str(int(submi))][0])
            correo_remitente.append(lista_usuario[str(int(submi))][1])
            organizacion_remitente.append(lista_usuario[str(int(submi))][2])
          except:
            nombre_remitente.append("")
            correo_remitente.append("")
            organizacion_remitente.append("")
    


    data['Nombre del solicitante']=nombre_solicitante
    data['Correo del solicitante']=correo_solicitante
    data['Organización del solicitante']=organizacion_solicitante

    data['Nombre del agente']=nombre_agente
    data['Correo del agente']=correo_agente
    data['Organización del agente']=organizacion_agente

    data['Nombre del remitente']=nombre_remitente
    data['Correo del remitente']=correo_remitente
    data['Organización del remitente']=organizacion_remitente

    # conversión de las columnas
    cl=data.columns.to_list()
    
    for col in cl:
      try:
        if int(col):
          field=lista_campos[int(col)]
          if field[1]=="tagger":
            for val in field[2]:
              data[col]=data[col].replace(val[1],val[0])
          elif field[1]=="multiselect":
            for val in field[2]:
              data[col]=data[col].replace(val[1],val[0])
          elif field[1]=="checkbox":
            data[col]=data[col].replace(None,False)
          else:
            True
          data.rename(columns = {col:field[0]}, inplace = True)
      except:
        True
    brands=[]
    for brand in data['brand_id']:
      try:
        brands.append(lista_marca[int(brand)])
      except:
        brands.append("")
    data['marca']=brands

    #se agrega grupo de agente asignado
    assig_group=[]
    for group in data['group_id']:
      try:
        assig_group.append(lista_grupo[int(group)])
      except:
        assig_group.append("")
    data['grupo agente']=assig_group
    #Se elimina las columnas innecesarias
    print("--Estoy eliminando columnas innecesarias 😩")
    data=data.drop(columns=['via',"forum_topic_id","custom_fields","fields","index"])
    #Estoy actualizando los nombres de las columnas
    print("--Cambio el nombre de las columnas 🤤")
    data.rename(columns = {"type":"tipo","subject":"asunto","description":"descripcion","priority":"prioridad","status":"estado","recipient":"casilla receptora","requester_id":"id solicitante","submitter_id":"id remitente","assignee_id":"id agente asignado","organization_id":"id organizacion solicitante","group_id":"id grupo asignado","ticket_form_id":"id formulario"}, inplace = True)
    #Crear archivo
    data.to_csv(carpeta_contenedora+'/'+start_time+".csv")
    list_csv.append(start_time+".csv")
    
    if(cont_1):
      cont=False
    start_time=str(new_start)
    N_1=N_1+1
    print("--Lista la transformación 😉")

  
  os.chdir(carpeta_contenedora)
  file_extension = '.csv'
  all_filenames = [i for i in glob.glob(f"*{file_extension}")]
  combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])


  
  combined_csv_data = combined_csv_data.loc[:, ~combined_csv_data.columns.str.startswith('Unnamed')]
  combined_csv_data.to_excel("01.xlsx")
  for file in list_csv:
   os.remove(carpeta_contenedora+"/"+file)
  print("Ya se creo el archivo 01.xlsx 🤤")

  return combined_csv_data
  



In [ ]:
df_etl=etl_ticket(z_domain,start_time,headers,carpeta_contenedora)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_etl

In [ ]:
a,b,c=data_ticket(z_domain,"1659916800",headers)
lista_campos=generarDataSetCamposTicket(z_domain,start_time,headers)
lista_usuario=data_user(z_domain,start_time,headers)
    

In [ ]:
combined_csv_data=a

lista_usuario=data_user(z_domain,start_time,headers)
nombre_solicitante=[]
correo_solicitante=[]
organizacion_solicitante=[]
nombre_agente=[]
correo_agente=[]
organizacion_agente=[]
nombre_remitente=[]
correo_remitente=[]
organizacion_remitente=[]

#solicitante_____________________________________________________
for req in combined_csv_data['requester_id']:
  if np.isnan(req):
    nombre_solicitante.append("")
    correo_solicitante.append("")
    organizacion_solicitante.append("")
  else:
    try:
      nombre_solicitante.append(lista_usuario[str(req)][0])
      correo_solicitante.append(lista_usuario[str(req)][1])
      organizacion_solicitante.append(lista_usuario[str(req)][2])
    except:
      nombre_solicitante.append("")
      correo_solicitante.append("")
      organizacion_solicitante.append("")

#Agente_____________________________________________________
for agent in combined_csv_data['assignee_id']:
  if np.isnan(agent):
    nombre_agente.append("")
    correo_agente.append("")
    organizacion_agente.append("")
  else:
    try:
      nombre_agente.append(lista_usuario[str(agent)][0])
      correo_agente.append(lista_usuario[str(agent)][1])
      organizacion_agente.append(lista_usuario[str(agent)][2])
    except:
      nombre_agente.append("")
      correo_agente.append("")
      organizacion_agente.append("")

#remitente_____________________________________________________
for submi in combined_csv_data['submitter_id']:
    if np.isnan(submi):
      nombre_remitente.append("")
      correo_remitente.append("")
      organizacion_remitente.append("")
    else:
      try:
        nombre_remitente.append(lista_usuario[str(submi)][0])
        correo_remitente.append(lista_usuario[str(submi)][1])
        organizacion_remitente.append(lista_usuario[str(submi)][2])
      except:
        nombre_remitente.append("")
        correo_remitente.append("")
        organizacion_remitente.append("")


combined_csv_data['Nombre del solicitante']=nombre_solicitante
combined_csv_data['Correo del solicitante']=correo_solicitante
combined_csv_data['Organización del solicitante']=organizacion_solicitante

combined_csv_data['Nombre del agente']=nombre_agente
combined_csv_data['Correo del agente']=correo_agente
combined_csv_data['Organización del agente']=organizacion_agente

combined_csv_data['Nombre del remitente']=nombre_remitente
combined_csv_data['Correo del remitente']=correo_remitente
combined_csv_data['Organización del remitente']=organizacion_remitente

In [ ]:
lista_usuario["423691799452"][1]